In [174]:
import voxel_slam

In [175]:
import open3d as o3d
import numpy as np
import copy 
import time

from sklearn.cluster import AgglomerativeClustering
from distinctipy import distinctipy

from voxel_slam.model import VoxelGrid
from voxel_slam.model import PCDPlane 

__all__ = ['VoxelFeatureMap']


# TODO: remove
def custom_gen_unique_colors(number_of_colors):
    np.random.seed(42)
    unique_color_set = set()
    while len(unique_color_set) < number_of_colors:
        unique_color_set.add(tuple(np.random.rand(1, 3)[0]))
    return list(unique_color_set)


class VoxelFeatureMap:
    # unique_colors = distinctipy.get_colors(300, rng=42)
    unique_colors = custom_gen_unique_colors(300)
    def __init__(self, clouds, poses, voxel_size):
        self._transformed_clouds = [copy.deepcopy(pcd).transform(pose) for pcd, pose in zip(clouds, poses)]
        self._voxel_to_pose_points_map = self.build_voxel_map_(voxel_size=voxel_size)

    @property
    def transformed_clouds(self):
        return copy.deepcopy(self._transformed_clouds)

    @staticmethod
    def find_cloud_bounds(clouds):
        min_bound = np.full(3, 1e9)
        max_bound = np.full(3, -1e9)
        for pcd in clouds:
            min_bound = np.minimum(min_bound, pcd.get_min_bound())
            max_bound = np.maximum(max_bound, pcd.get_max_bound())
        return min_bound, max_bound

    def build_voxel_map_(self, voxel_size):
        voxel_grid = VoxelGrid(*VoxelFeatureMap.find_cloud_bounds(self._transformed_clouds), voxel_size=voxel_size)
        voxel_to_pose_points_map = {}

        for pose_id, pcd in enumerate(self._transformed_clouds):
            for point_id, point in enumerate(np.asarray(pcd.points)):
                if not np.any(point):
                    continue

                voxel_center = voxel_grid.get_voxel_index(point)
                if voxel_center not in voxel_to_pose_points_map:
                    voxel_to_pose_points_map[voxel_center] = {}
                
                voxel_pose_points = voxel_to_pose_points_map[voxel_center].get(pose_id, PCDPlane(points=[], pcd_idx=[]))
                voxel_pose_points.add_point(point, point_id)
                voxel_to_pose_points_map[voxel_center].update({pose_id: voxel_pose_points})
              
        return voxel_to_pose_points_map
    
    def extract_voxel_features(self, ransac_distance_threshold, points_filter_function=lambda feature_points: True):
        voxel_feature_map = {voxel_id: {} for voxel_id in self._voxel_to_pose_points_map.keys()}

        for voxel_id, pose_to_points in self._voxel_to_pose_points_map.items():
            for pose_id, pcd_plane in pose_to_points.items():
                try:
                    max_plane = pcd_plane.segment_max_plane(ransac_distance_threshold)
                except RuntimeError:
                    continue

                if points_filter_function(np.asarray(max_plane.points)):
                    voxel_feature_map[voxel_id][pose_id] = max_plane

        return voxel_feature_map

    @staticmethod
    def filter_voxel_features(voxel_feature_map, cosine_distance_threshold=0.4, preserve_non_informative_voxels=False):
        non_informative_voxels = []

        for voxel_id, pose_to_points in voxel_feature_map.items():
            normals = [plane.get_plane_equation()[:-1] for plane in pose_to_points.values()]
            if len(normals) < 2:
                if not preserve_non_informative_voxels:
                    non_informative_voxels.append(voxel_id)
                continue
            
            clustering = AgglomerativeClustering(
                n_clusters=None,
                distance_threshold=cosine_distance_threshold,
                metric="cosine",
                linkage="single",
                compute_distances=True 
            ).fit(np.asarray(normals))

            stable_plane_label = np.bincount(clustering.labels_).argmax()
            outlier_plane_poses = np.asarray(list(pose_to_points.keys()))[clustering.labels_ != stable_plane_label]
            for pose_id in outlier_plane_poses:
                pose_to_points.pop(pose_id)
            
            # TODO: Unite with previous steps
            # Add filter by D
            planes_d = [plane.get_plane_equation()[-1] for plane in pose_to_points.values()]
            if len(planes_d) < 2:
                if not preserve_non_informative_voxels:
                    non_informative_voxels.append(voxel_id)
                continue

        # Remove voxels that cover less than two poses
        for voxel_id in non_informative_voxels:
            voxel_feature_map.pop(voxel_id)
    
    def get_colored_feature_clouds(self, voxel_feature_map, color_method="pose"):
        time_init_start = time.perf_counter_ns()
        allowed_methods = ["pose", "voxel"]
        if color_method not in allowed_methods:
            raise TypeError(f"Color method has to be one of {'|'.join(allowed_methods)}")
        colored_clouds = self.transformed_clouds

        print("Time (init):", (time.perf_counter_ns() - time_init_start) / 1e9)  

        color_to_voxel_center = {}

        for voxel_id, (voxel_center, pose_to_points) in enumerate(voxel_feature_map.items()):
            for pose_id, pcd_plane in pose_to_points.items():
                cloud_colors = np.asarray(colored_clouds[pose_id].colors)
                if color_method == "voxel":
                    cloud_colors[pcd_plane.pcd_idx] = self.unique_colors[voxel_id]
                    color_to_voxel_center[self.unique_colors[voxel_id]] = voxel_center
                elif color_method == "pose":
                    cloud_colors[pcd_plane.pcd_idx] = self.unique_colors[pose_id]
                colored_clouds[pose_id].colors = o3d.utility.Vector3dVector(cloud_colors)
        
        return colored_clouds, color_to_voxel_center

In [197]:
def is_good_small(plane_points):
    c = np.mean(plane_points, axis=0)
    A = np.array(plane_points) - c
    eigvals, _ = np.linalg.eig(A.T @ A)
    eigvals.sort()
    # print(eigvals)
    k = 5
    return eigvals[2] < eigvals[1] * k and plane_points.shape[0] > 50

In [180]:
import random
from tqdm import tqdm

In [198]:
def check_planes(plane_normals): 
    matrix = [] 
    for plane in plane_normals: 
        matrix.append(plane) 
    matrix = np.asarray(matrix) 
    covarience = matrix.T @ matrix 
    eigvals, eigvects = np.linalg.eig(covarience) 
    return eigvals

def get_optimized(clouds_cp, poses_cp):
    clouds = copy.deepcopy(clouds_cp)
    poses = copy.deepcopy(poses_cp)
    voxel_map = VoxelFeatureMap(clouds, poses, voxel_size=2)
    f_function = lambda points: is_good_small(np.asarray(points))
    
    voxel_feature_map = voxel_map.extract_voxel_features(
        ransac_distance_threshold=0.005, 
        points_filter_function=f_function
    )
    
    voxel_map.filter_voxel_features(voxel_feature_map)
    
    N = len(clouds)
    non_informative_voxels = []
    for voxel_id in voxel_feature_map:
        if len(voxel_feature_map[voxel_id]) < N:
            non_informative_voxels.append(voxel_id)

    for voxel_id in non_informative_voxels:
        voxel_feature_map.pop(voxel_id)
    
    voxel_pcd, _ = voxel_map.get_colored_feature_clouds(voxel_feature_map, color_method="voxel")
    
    voxel_indices = list(voxel_feature_map.keys())
    print(len(voxel_indices))

    chis = []
    poses_combo = []
    voxel_ids = []
    for _ in range(30):
        voxel_map_cut = {}
        vox_ind = random.sample(voxel_indices, int(0.7 * len(voxel_indices)))
        planes_list = []
        for k in vox_ind:
            voxel_map_cut[k] = copy.deepcopy(voxel_feature_map[k])
            planes_list.append(voxel_map_cut[k][0].get_plane_equation()[:-1])
        
        if min(check_planes(planes_list)) < 1e-1:
            # print('Planes filter')
            continue
            
        graph = voxel_slam.backend.FGraph(voxel_map_cut, number_of_poses=len(poses))
        optimized_poses, converged = graph.get_optimized_poses(number_of_iterations=1000, verbose=False)
        # print(graph.graph.chi2(), converged)

        if not converged:
            continue
        
        chis.append(graph.graph.chi2())
        # print(chis[-1])
        poses_combo.append(optimized_poses)
        voxel_ids.append(vox_ind)
        
    minind = np.argmin(np.array(chis))
    perfect_poses = poses_combo[minind]
    print(min(chis))

    voxel_map_cut = {}
    for k in voxel_ids[minind]:
        voxel_map_cut[k] = copy.deepcopy(voxel_feature_map[k])
        
    voxel_pcd, _ = voxel_map.get_colored_feature_clouds(voxel_map_cut, color_method="voxel")
    # perfect_poses = []
    
    return voxel_pcd, perfect_poses, voxel_map_cut

In [199]:
import open3d as o3d 
import numpy as np

import pathlib

poses_path = "/home/hedgehog/Downloads/data/poses"
clouds_path = "/home/hedgehog/Downloads/data/clouds"

clouds = []
poses = []

for pose_file in sorted(pathlib.Path(poses_path).iterdir(), key=lambda x: int(x.stem)):
    pose_id = pose_file.stem
    clouds.append(o3d.io.read_point_cloud(
        clouds_path + "/" + pose_id + ".pcd"
    ))
    poses.append(np.genfromtxt(pose_file))

In [200]:
window_size = 5
optimized_submaps = {}

transformed_clouds = voxel_slam.VoxelFeatureMap(clouds, poses, voxel_size=2).transformed_clouds

for i in tqdm(range(0, len(clouds), window_size)):
    optimized_submaps[i] = get_optimized(transformed_clouds[i:i+window_size], [np.eye(4)] * window_size)

  0%|                                                                                                                                                                                | 0/6 [00:00<?, ?it/s]

Time (init): 0.00120164
19


 17%|████████████████████████████                                                                                                                                            | 1/6 [00:04<00:22,  4.58s/it]

0.12796160269193124
Time (init): 0.001025133
Time (init): 0.001036653
19


 33%|████████████████████████████████████████████████████████                                                                                                                | 2/6 [00:10<00:21,  5.41s/it]

...

0.9988614663817249
Time (init): 0.001440634
Time (init): 0.00218101
22


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                            | 5/6 [00:44<00:10, 10.61s/it]

1.12648076582344
Time (init): 0.001848807
Time (init): 0.002882895
21


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:59<00:00,  9.87s/it]

4.221030264240235
Time (init): 0.002751169


In [203]:
aggregated_submaps = {k: voxel_slam.aggregate_map(pcd, pose) for k, (pcd, pose, _) in optimized_submaps.items()}

In [204]:
for k, pcd in aggregated_submaps.items():
    o3d.visualization.draw_geometries([pcd])